***
## Проблема лишних запросов

В Django ORM предусмотрен простой способ обращения к связанным объектам. Можно запросить атрибут объекта, ссылающийся на другую модель, и прямо в HTML-шаблоне получить значения полей связанного объекта — через точечную нотацию. Попробуем решить поставленную задачу именно этим способом.

Во view-функции `index()` приложения homepage получим QuerySet с объектами `IceCream` и передадим его в HTML-шаблон:

```py
# homepage/views.py

...

def index(request):
    template_name = 'homepage/index.html'

    ice_cream_list = IceCream.objects.all()

    context = {
        'ice_cream_list': ice_cream_list,
    }
    return render(request, template_name, context) 
```

В объектах модели `IceCream` есть атрибут `category`, ссылающийся на объект модели `Category`, а у объектов модели `Category` есть атрибут `title`. 

Выведем название категории в шаблон через точечную нотацию — `ice_cream.category.title`:

```html
<!-- templates/homepage/index.html -->

...

{% block content %}
  <h1 class="pb-2 mb-0">Главная страница</h1>
  
  {% for ice_cream in ice_cream_list %}  
    <h3>{{ ice_cream.title }} ID: {{ ice_cream.id }}</h3>
    <!-- Выведем название категории -->
    <p>Категория: {{ ice_cream.category.title }}</p>
  {% endfor %}
{% endblock %}
```

***
## JOIN c помощью метода .values()

Метод `.values()` может вернуть не только поля запрошенной модели, но и значения полей модели, которая связана с запрошенной:

```py
ice_cream_list = IceCream.objects.values('id', 'title', 'category__title')
# values(..., '<поле fk>__<поле в модели, связанной по fk>') 
```

В аргументе метода `.values()` передаётся имя атрибута, где хранится внешний ключ (`category` в приведённом примере), и через двойное нижнее подчёркивание — название того поля связанной модели, значение которого нужно получить.

Подобный синтаксис применяется и в шаблоне для вывода значения поля из связанной модели:

```html
{% for ice_cream in ice_cream_list %}
  <h3>{{ ice_cream.title }} ID: {{ ice_cream.id }}</h3>
  <p>Категория: {{ ice_cream.category__title }}</p>
{% endfor %}  
```

Сгенерированная веб-страница ничем не будет отличаться от предыдущего варианта.

> Важный момент: метод `.values()` возвращает список словарей, а не объектов. На практике это означает, что при использовании метода `.values() `в ORM-запросе обратиться к значению связанного поля из шаблона через точечную нотацию не получится. В этом случае нужно использовать синтаксис с двойным подчёркиванием, например, вот так: `{{ ice_cream.category__title }}`.

***
## JOIN c помощью .select_related()

Другой способ создать JOIN-запрос в Django ORM — вызвать метод `.select_related()`.

Вот его синтаксис:

```py
ice_cream_list = IceCream.objects.select_related('category') 
```

Соответствующий ему SQL запрос тогда будет таким:

```sql
SELECT "ice_cream_icecream"."id",
       "ice_cream_icecream"."is_published",
       "ice_cream_icecream"."is_on_main",
       "ice_cream_icecream"."title",
       "ice_cream_icecream"."description",
       "ice_cream_icecream"."wrapper_id",
       "ice_cream_icecream"."category_id",
       "ice_cream_category"."id",
       "ice_cream_category"."is_published",
       "ice_cream_category"."title",
       "ice_cream_category"."slug",
       "ice_cream_category"."output_order"
FROM "ice_cream_icecream"
INNER JOIN "ice_cream_category"
    ON ("ice_cream_icecream"."category_id" = "ice_cream_category"."id")
```

В отличие от `.values()`, метод `.select_related()` возвращает QuerySet со **списком объектов**, в которых содержатся **все поля** связанных моделей.

В шаблоне можно получить доступ к полям связанного объекта через точечную нотацию:

```html
{% for ice_cream in ice_cream_list %}
  <!-- Выводим id и title объекта IceCream -->
  <h3>{{ ice_cream.title }} ID: {{ ice_cream.id }}</h3> 
  ...
  <!-- Выводим поля связанного объекта Category -->
  <p>Категория: {{ ice_cream.category.title }}</p>
  <p>Слаг категории: {{ ice_cream.category.slug }}</p>
  <p>Порядок отображения: {{ ice_cream.category.output_order }}</p>
{% endfor %}  
```

***
## Фильтры в связанных моделях

Итоговую выборку, получившуюся при запросе к связанным моделям, можно фильтровать по полям связанной модели. 

Например, если в проекте «Анфиса для друзей» какая-то категория снята с публикации (`is_published=False`) — должны быть скрыты и все сорта мороженого, связанные с этой категорией. 

Выборка фильтруется через метод `.filter()` точно так же, как и при запросах к отдельной модели. Единственная разница состоит в том, что название поля связанной модели указывается через двойное подчёркивание после названия поля FK основной модели (в нашем случае Foreign Key хранится в поле `category`).

Фильтрация при работе с методом `.values()`:

```py
ice_cream_list = IceCream.objects.values(
    'id', 'title', 'category__title'
).filter(
    # Вернуть только те объекты IceCream, у которых
    # в связанном объекте Category в поле is_published хранится значение True:
    category__is_published=True
) 
```

Фильтрация при работе с методом `.select_related()`: 

```py
ice_cream_list = IceCream.objects.select_related(
    'category'
).filter(
    # В точности то же самое:
    category__is_published=True
) 
```

Оба этих варианта добавят в запрос условие `WHERE`:

```sql
...
WHERE ("ice_cream_category"."is_published") 
```

Для моделей, связанных «один к одному», методы `.values()` и `.select_related()` работают так же, как и для «многие к одному».